<a href="https://colab.research.google.com/github/vifirsanova/hse-python-course/blob/main/compression/quantization_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install -q tensorflow-model-optimization

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 12.2 MB/s eta 0:00:00


In [5]:
import tensorflow as tf
from tensorflow.keras import layers, models
import tensorflow_model_optimization as tfmot

In [6]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [9]:
from tensorflow_model_optimization.python.core.keras.compat import keras

model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

In [10]:
# квантизация модели для обучения
quantize_model = tfmot.quantization.keras.quantize_model
model = quantize_model(model)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
model.fit(x_train, y_train, epochs=5, validation_split=0.2)

Epoch 1/5
1500/1500 [==============================] - 55s 35ms/step - loss: 0.1615 - accuracy: 0.9506 - val_loss: 0.0628 - val_accuracy: 0.9826
Epoch 2/5
1500/1500 [==============================] - 53s 35ms/step - loss: 0.0524 - accuracy: 0.9840 - val_loss: 0.0451 - val_accuracy: 0.9866
Epoch 3/5
1500/1500 [==============================] - 51s 34ms/step - loss: 0.0359 - accuracy: 0.9885 - val_loss: 0.0449 - val_accuracy: 0.9864
Epoch 4/5
1500/1500 [==============================] - 60s 40ms/step - loss: 0.0273 - accuracy: 0.9913 - val_loss: 0.0489 - val_accuracy: 0.9868
Epoch 5/5
1500/1500 [==============================] - 49s 33ms/step - loss: 0.0202 - accuracy: 0.9931 - val_loss: 0.0377 - val_accuracy: 0.9892


In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_layer (QuantizeLa  (None, 28, 28, 1)         3         
 yer)                                                            
                                                                 
 quant_conv2d (QuantizeWrap  (None, 26, 26, 32)        387       
 perV2)                                                          
                                                                 
 quant_max_pooling2d (Quant  (None, 13, 13, 32)        1         
 izeWrapperV2)                                                   
                                                                 
 quant_conv2d_1 (QuantizeWr  (None, 11, 11, 64)        18627     
 apperV2)                                                        
                                                                 
 quant_max_pooling2d_1 (Qua  (None, 5, 5, 64)          1

In [13]:
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_accuracy}')

313/313 [==============================] - 4s 13ms/step - loss: 0.0312 - accuracy: 0.9902
Test accuracy: 0.9901999831199646


In [14]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_tflite_model = converter.convert()

/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/convert.py:983: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
